In [ ]:
import csv
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Genre Identification Dataset

In [ ]:
import pandas as pd
genre_created = pd.read_json('gdrive/MyDrive/CIS 519 Project/Data/Raw Data/goodreads_book_genres_initial.json.gz',lines=True)
genre_created.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [ ]:
dict(genre_created.loc[3,'genres'])

{'fiction': 555, 'mystery, thriller, crime': 10, 'romance': 23}

In [ ]:
t_dict = genre_created.loc[3,'genres']
top_three_genre = sorted(t_dict, key=t_dict.get, reverse=True)


{'Second': 'romance', 'Third': 'mystery, thriller, crime', 'Top': 'fiction'}

In [ ]:
t2 = ['hi']
t2.append('')
print(t2)

['hi', '']


In [ ]:
def cleanGenreList(originalList):
  out = sorted(dict(originalList['genres']), key=originalList['genres'].get, reverse=True)
  if len(out) == 0:
    out =  ['','','']
  elif len(out) == 1:
    out.append('')
    out.append('')
  elif len(out) == 2:
    out.append('')
  else:
    out = out[:3]
  return out


In [ ]:
genre_created_cleaned = genre_created.copy()
genre_created_cleaned[['first', 'second', 'third']] = genre_created.apply(lambda x: cleanGenreList(x), axis=1, result_type='expand')
genre_created_cleaned.head()

,book_id,genres,first,second,third
0,5333265,"{'history, historical fiction, biography': 1}","history, historical fiction, biography",,
1,1333909,"{'fiction': 219, 'history, historical fiction,...",fiction,"history, historical fiction, biography",
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys...","fantasy, paranormal",fiction,"mystery, thriller, crime"
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri...",fiction,romance,"mystery, thriller, crime"
4,287140,{'non-fiction': 3},non-fiction,,


In [ ]:
def cleanGenre(row):
  if row['first'] == 'fiction' and row['second'] != '':
    return row['second']
  else:
    return row['first']

In [ ]:
genre_created_cleaned["genre_cleaned"] = genre_created_cleaned.apply(lambda x: cleanGenre(x), axis=1)
genre_created_cleaned.head()

,book_id,genres,first,second,third,genre_cleaned
0,5333265,"{'history, historical fiction, biography': 1}","history, historical fiction, biography",,,"history, historical fiction, biography"
1,1333909,"{'fiction': 219, 'history, historical fiction,...",fiction,"history, historical fiction, biography",,"history, historical fiction, biography"
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys...","fantasy, paranormal",fiction,"mystery, thriller, crime","fantasy, paranormal"
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri...",fiction,romance,"mystery, thriller, crime",romance
4,287140,{'non-fiction': 3},non-fiction,,,non-fiction


In [ ]:
genre_created_cleaned.groupby(by=['genre_cleaned']).count()

,book_id,genres,first,second,third
genre_cleaned,,,,,
,409513,409513,409513,409513,409513
children,129873,129873,129873,129873,129873
"comics, graphic",93720,93720,93720,93720,93720
"fantasy, paranormal",299786,299786,299786,299786,299786
fiction,83201,83201,83201,83201,83201
"history, historical fiction, biography",296874,296874,296874,296874,296874
"mystery, thriller, crime",243219,243219,243219,243219,243219
non-fiction,346963,346963,346963,346963,346963
poetry,50070,50070,50070,50070,50070


In [ ]:
genre_created_cleaned.to_csv('gdrive/MyDrive/CIS 519 Project/Data/genre_lookup_cleaned.csv')
#genre_lookup = pd.read_csv('gdrive/MyDrive/CIS 519 Project/Data/genre_lookup_cleaned.csv')

# Reading in Dataset from JSON

In [ ]:
import json
import gzip
import csv

fields_to_take = ['isbn', 'average_rating', 'description', 'link', 'authors',
                  'publisher', 'num_pages', 'isbn13', 'publication_year', 'image_url', 
                  'book_id', 'title', 'title_without_series', 'language_code']
dict_chuck = []

i = 0
first = True
for line in gzip.open("gdrive/MyDrive/CIS 519 Project/Data/Raw Data/goodreads_books.json.gz", 'r'):
  json_line = json.loads(line.decode("utf-8"))
  line_dict = {}
  keep = False
  for field in fields_to_take:
    if field == "authors":
      if json_line[field] == []:
        out = ''
      else:
        out = json_line[field][0]['author_id']
    elif field == "image_url" and json_line[field] == 'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png':
      out = ''
    elif field == "language_code":
      if json_line[field] == "eng" or json_line[field] == "":
        keep = True
        out = json_line[field]
    else:
      out = json_line[field]

    line_dict[field] = out
  if keep: 
    dict_chuck.append(line_dict)

  i+=1

  # append rows every 5000 lines
  if i % 10000 == 0:
    print(i)
    if first:
      print("FIRST")
      with open('gdrive/MyDrive/CIS 519 Project/Data/goodreads_books_cleaned.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = fields_to_take)
        writer.writeheader()
        writer.writerows(dict_chuck)
      first = False
      dict_chuck = []
    else:
      with open('gdrive/MyDrive/CIS 519 Project/Data/goodreads_books_cleaned.csv', 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = line_dict.keys())
        writer.writerows(dict_chuck)
      dict_chuck = []

# cut off run after 10,000 rows
  #if i == 20001:
  #  print(1/0)



In [ ]:
import pandas as pd
df = pd.read_csv('gdrive/MyDrive/CIS 519 Project/Data/goodreads_books_cleaned.csv')
df.head()

,isbn,average_rating,description,link,authors,publisher,num_pages,isbn13,publication_year,image_url,book_id,title,title_without_series,language_code
0,0312853122,4.00,NaN,https://www.goodreads.com/book/show/5333265-w-...,604031.0,St. Martin's Press,256.0,9780312853129,1984.0,https://images.gr-assets.com/books/1310220028m...,5333265,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film,NaN
1,0743509986,3.23,"Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,626222.0,Simon & Schuster Audio,NaN,9780743509985,2001.0,NaN,1333909,Good Harbor,Good Harbor,NaN
2,NaN,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,10333.0,"Nelson Doubleday, Inc.",600.0,NaN,1987.0,https://images.gr-assets.com/books/1304100136m...,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",eng
3,0743294297,3.49,Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,9212.0,Atria Books,368.0,9780743294294,2009.0,NaN,6066819,Best Friends Forever,Best Friends Forever,eng
4,0850308712,3.40,NaN,https://www.goodreads.com/book/show/287140.Run...,149918.0,NaN,NaN,9780850308716,NaN,https://images.gr-assets.com/books/1413219371m...,287140,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...,NaN


In [ ]:
df.shape

(1768097, 14)

# Joined together

In [ ]:
df_w_genre = df.merge(genre_lookup, how="inner", on="book_id")
df_w_genre = df_w_genre.drop(columns=['Unnamed: 0', 'genres'])
df_w_genre.head()

,isbn,average_rating,description,link,authors,publisher,num_pages,isbn13,publication_year,image_url,book_id,title,title_without_series,language_code,first,second,third,genre_cleaned
0,0312853122,4.00,NaN,https://www.goodreads.com/book/show/5333265-w-...,604031.0,St. Martin's Press,256.0,9780312853129,1984.0,https://images.gr-assets.com/books/1310220028m...,5333265,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film,NaN,"history, historical fiction, biography",NaN,NaN,"history, historical fiction, biography"
1,0743509986,3.23,"Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,626222.0,Simon & Schuster Audio,NaN,9780743509985,2001.0,NaN,1333909,Good Harbor,Good Harbor,NaN,fiction,"history, historical fiction, biography",NaN,"history, historical fiction, biography"
2,NaN,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,10333.0,"Nelson Doubleday, Inc.",600.0,NaN,1987.0,https://images.gr-assets.com/books/1304100136m...,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",eng,"fantasy, paranormal",fiction,"mystery, thriller, crime","fantasy, paranormal"
3,0743294297,3.49,Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,9212.0,Atria Books,368.0,9780743294294,2009.0,NaN,6066819,Best Friends Forever,Best Friends Forever,eng,fiction,romance,"mystery, thriller, crime",romance
4,0850308712,3.40,NaN,https://www.goodreads.com/book/show/287140.Run...,149918.0,NaN,NaN,9780850308716,NaN,https://images.gr-assets.com/books/1413219371m...,287140,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...,NaN,non-fiction,NaN,NaN,non-fiction


In [ ]:
df_w_genre.to_csv('gdrive/MyDrive/CIS 519 Project/Data/books_with_genre.csv.gz', compression='gzip')

# Cleaning

## Cleaning Language

In [ ]:
def detectLang(x):
  if x != x:
    return ''
  elif type(x) == str:
    try:
      return detect(x)
    except:
      print("!!!", x)
  else:
    print(type(x))


In [ ]:
df_w_genre['detected_lang'] = df_w_genre.loc[(df_w_genre['language_code'] != df_w_genre['language_code']) & (df_w_genre['description'] == df_w_genre['description']) & (~df_w_genre['description'].isin(['<>', '<', '>', '.', ','])), 'description'].apply(detectLang)



!!! http://www.cartea-prin-posta.ro/carte...
!!! http://ahmadheydarbeygi.wordpress.com/
!!! "
!!! <>
https://pt.wikipedia.org/wiki/Little_...
!!! http://lackingtons.com/2015/07/29/the...
!!! http://www.harpercollins.co.nz/extra-...
!!! https://drive.google.com/file/d/0B4CS...
!!! http://muumuuhouse.com/dou.fiction2.html
!!! ###############################################################################################################################################################################################################################################################
!!! http://www.4shared.com/file/EGWfHlRX/...
!!! http://saaid.net/Doat/alhaweel/61.pdf
!!! http://archive.org/download/b4a0038/b...
!!! 
!!! <>.
!!! https://www.mediafire.com/?drmlpd26dd...
!!! ###############################################################################################################################################################################################################################

In [ ]:
from collections import Counter

Counter(df_w_genre['detected_lang'])

Counter({None: 145,
         'af': 129,
         'ca': 135,
         'cs': 170,
         'cy': 4343,
         'da': 347,
         'de': 2877,
         'en': 754055,
         'es': 7630,
         'et': 284,
         'fi': 424,
         'fr': 4041,
         'hr': 361,
         'hu': 85,
         'id': 1563,
         'it': 1652,
         'lt': 98,
         'lv': 40,
         nan: 984276,
         'nl': 809,
         'no': 189,
         'pl': 353,
         'pt': 1454,
         'ro': 277,
         'sk': 184,
         'sl': 418,
         'so': 303,
         'sq': 108,
         'sv': 431,
         'sw': 66,
         'tl': 156,
         'tr': 537,
         'vi': 157})

Most of the nans in this list were values that were assigned English by the dataset. The values of None are the values that were exceptions in the language assignment code so we want to remove those.

In [ ]:
df_w_genre = df_w_genre[~df_w_genre['detected_lang'].isin([None])]
df_w_genre = df_w_genre[df_w_genre['first'] == df['first']]

## Assigning other Genre Options

In [ ]:
def cleanGenre2(row):
  if row['first'] == 'fiction' and row['second'] == row['second']:
    return (row['first']+": "+row['second'])
  else:
    return row['first']


df_w_genre['genre_cleaned2'] = df_w_genre.apply(cleanGenre2, axis=1)

In [ ]:
def cleanGenre3(row):
  if row['second'] == 'fiction':
    return (row['second']+": "+row['first'])
  else:
    return row['genre_cleaned2']


df_w_genre['genre_cleaned3'] = df_w_genre.apply(cleanGenre3, axis=1)

In [ ]:
df_w_genre = df[(df['language_code'] == 'eng') | (df['detected_lang'] == 'en') | ((df['description'] != df['description']) & (df['detected_lang'] != df['detected_lang'] ))]
df_w_genre = df.drop(columns=['language_code', 'detected_lang', 'Unnamed: 0'])
df_w_genre.to_csv('gdrive/MyDrive/CIS 519 Project/Data/books_clean.csv.gz', compression='gzip')
